In [1]:
import os
import sys
from supportFunctions import *

root_folder = "..\\data\\NB15\\flows\\"

In [2]:
#### PARAMETERS #####

## Generic parameters
temporal = True # used to determine if this evaluation assumes a "temporal" dependency among its samples
base_clf = 'dt' # name of the classifier used for this "run". Available names: ['dt', 'rf', 'hgb', 'lr']. You can add more by editing the supportFunctions file
test_size = 0.2 # proportion of the dataset used for testing. We always kept it fixed to 0.2 for our paper
train_size = 100 # proportion of the REMAINING data that are used for training (if >1, then it will take that exact amount). To reproduce the results of the paper, use: 100 (for "limited" training data) or 0.2 or 0.5 or 0.99 (for scarce, moderate, abundant training data, respectively) 
agreement = 0.5 # from 0 to 1. Proportion of classifiers that must agree on an attack (for the ensemble). This is fixed in our paper.
max_size = 500000 ## maximum amount of samples to include when creating the initial dataframes. This is fixed in our paper
max_size_atk = int(max_size / 3) # maximum amount of malicious samples per class. This is fixed in our paper

## Adversarial Attacks parameters
atk_intensity = 1 # send 100 packets, each of 100 bytes, over 100 seconds
pkt_intensity = atk_intensity * 100 # 
byt_intensity = pkt_intensity * 100 # 
dur_intensity = atk_intensity * 100 # consider seconds

In [3]:
### Reading input data

malicious_folder = root_folder + "malicious/"

benign_file = root_folder + "benign.csv"
benign_df = pd.read_csv(benign_file, header='infer', index_col=0)
benign_df = benign_df.sample(min(max_size, len(benign_df)))
#sort by timestamp
if temporal == True:
    pass # already sorted
benign_df.reset_index(inplace=True, drop=True)

# attack_names = ['expl', 'recon', 'dos', 'shell', 'fuzz', 'bdoor', 'ana', 'worm', 'other']
attack_names = ['expl', 'recon', 'dos', 'shell', 'fuzz', 'bdoor', 'ana']

expl_file = malicious_folder + "expl.csv"
recon_file = malicious_folder + "recon.csv"
dos_file = malicious_folder + "dos.csv"
shell_file = malicious_folder + "shell.csv"
fuzz_file = malicious_folder + "fuzz.csv"
worm_file = malicious_folder + "worm.csv" # excluded, only 174 samples
bdoor_file = malicious_folder + "bdoor.csv"
ana_file = malicious_folder + "ana.csv"
other_file = malicious_folder + "other.csv" # excluded due to mismatch



for a in attack_names:
    exec(f"{a}_df = pd.read_csv({a}_file, header='infer', index_col=0)")
    exec(f"{a}_df = {a}_df.sample(min(max_size_atk, len({a}_df)))")
    # sort by timestamp
    if temporal == True:
        pass
    exec(f"{a}_df.reset_index(inplace=True, drop=True)")
    exec(f"{a}_df['Label'] = a")

In [4]:
# Determining Train and Test sets for each class

df_list = [benign_df]
for a in attack_names:
    exec(f"df_list.append({a}_df)")

if temporal == True:
    for dummy_df in df_list:
        if train_size <=1:
            train_threshold = int(((1-test_size) * train_size) * len(dummy_df))
        else:
            train_threshold = int(100)
        test_threshold = len(dummy_df) - int(test_size * len(dummy_df))
        dummy_df['index'] = dummy_df.index
        dummy_df['is_test'] = np.where(dummy_df['index'] >= test_threshold , True, False)
        dummy_df['is_train'] = np.where(dummy_df['index'] <= train_threshold , True, False)
else:
    for dummy_df in df_list:
        if train_size <= 1:
            train_threshold = test_size + (1-test_size)*train_size
        else:
            train_threshold = test_size + ((train_size * 100) / (len(dummy_df)) / 100)       
        dummy_df['seed'] = (np.random.uniform(0,1,len(dummy_df)))
        dummy_df['is_test'] = np.where(dummy_df['seed'] <= test_size, True, False)
        dummy_df['is_train'] = np.where((dummy_df['seed'] <= train_threshold) & (dummy_df['is_test']==False), True, False)

# get all together
all_df = pd.concat(df_list)

In [5]:
def handle_categorical(df):
    ## Handling categorical data
    df_dummy = df.copy(deep=True)
    df_dummy['Nature'] = np.where(df_dummy['Label'].str.contains('BENIGN'),0,1)
    
    for column_name in df_dummy.columns:
        if column_name == ('SrcPort_type'):
            df_dummy[column_name+"-f"] = pd.factorize(df_dummy[column_name])[0]
        elif column_name == ('DstPort_type'):
            df_dummy[column_name+"-f"] = pd.factorize(df_dummy[column_name])[0]
        elif column_name == ('Proto'):
            df_dummy[column_name+"-f"] = pd.factorize(df_dummy[column_name])[0]
        elif column_name == ('State'):
            df_dummy[column_name+"-f"] = pd.factorize(df_dummy[column_name])[0]
        elif column_name == ('Service'):
            df_dummy[column_name+"-f"] = pd.factorize(df_dummy[column_name])[0]
        elif column_name == ('ct_ftp_cmd'):
            df_dummy[column_name+"-f"] = pd.factorize(df_dummy[column_name])[0]
        else:
            pass
    return df_dummy

all_df = handle_categorical(all_df)
all_df['Label_cat'] = pd.factorize(all_df['Label'])[0]

all_df['totPkt'] = all_df['SrcPkts'] + all_df['DstPkts']
all_df['totByt'] = all_df['SrcBytes'] + all_df['DstBytes']

all_df['int2int'] = np.where( ((all_df['SrcIP_internal']==True) & (all_df['DstIP_internal']==True)), True, False)
all_train, all_test = all_df[all_df['is_train']==True], all_df[all_df['is_test']==True]

### SPLITTING ALL BACK ####
benign_df = all_df[all_df['Label']=='BENIGN']

for a in attack_names:
    exec(f"{a}_df = all_df[all_df['Label']=='{a}']")
    
malicious_df = all_df[all_df['Label']!='BENIGN']
malicious_train, malicious_test = malicious_df[malicious_df['is_train']==True], malicious_df[malicious_df['is_test']==True]

benign_train = benign_df[benign_df['is_train']==True]
benign_test = benign_df[benign_df['is_test']==True]



print("& 0 & \\textit{{Benign}} & {} & \\\\ \\cline{{2-4}}".format(len(benign_df)))


for i,a in enumerate(attack_names):
    exec(f"print('& {i+1} & \\\\textit{{{{{a}}}}} & {{}} \\\\\\\\ \\\\cline{{{{2-4}}}}'.format(len({a}_df)))")

& 0 & \textit{Benign} & 500000 & \\ \cline{2-4}
& 1 & \textit{expl} & 44525 \\ \cline{2-4}
& 2 & \textit{recon} & 13987 \\ \cline{2-4}
& 3 & \textit{dos} & 16353 \\ \cline{2-4}
& 4 & \textit{shell} & 1511 \\ \cline{2-4}
& 5 & \textit{fuzz} & 24246 \\ \cline{2-4}
& 6 & \textit{bdoor} & 2329 \\ \cline{2-4}
& 7 & \textit{ana} & 2677 \\ \cline{2-4}


In [6]:
## Feature sets

# the following is the "complete" feature set

features = ['Duration', 'SrcBytes', 'DstBytes',
       'SrcTTL', 'DstTTL', 'SrcLoss', 'DstLoss', 'SrcBpS', 'DstBpS',
       'SrcPkts', 'DstPkts', 'SrcWin', 'DstWin', 'SrcTcpb', 'DstTcpb',
       'SrcMean', 'DstMean', 'Trans_Depth', 'res_bdy_len', 'SrcJit', 'DstJit',
       'SrcAit', 'DstAit', 'TcpRrt', 'SynAck',
       'AckDat', 'is_sm_ips_ports', 'ct_state_ttl', 'ct_flw_http_mthd',
       'is_ftp_login', 'ct_srv_src', 'ct_srv_dst', 'ct_dst_ltm',
       'ct_src_ltm', 'ct_src_dport', 'ct_dst_sport', 'ct_dst_src_ltm',
       'DstIP_internal', 'SrcIP_internal', 'Proto-f', 'State-f',
       'Service-f', 'ct_ftp_cmd-f', 'SrcPort_type-f', 'DstPort_type-f',
       ]

# this is for the "essential" feature set
small_features = ['Duration', 'SrcBytes', 'DstBytes',
       'SrcPkts', 'DstPkts', 'totPkt', 'totByt',
       'SynAck',
       'AckDat', 'is_sm_ips_ports', 'ct_state_ttl', 'ct_flw_http_mthd',
       'is_ftp_login', 'ct_srv_src', 'ct_srv_dst', 'ct_dst_ltm',
       'ct_src_ltm', 
       'int2int',
        'Proto-f', 'State-f',
       'Service-f', 'ct_ftp_cmd-f', 'SrcPort_type-f', 'DstPort_type-f',]

In [7]:
# creating adversarial dataset
mal_base = malicious_df[((malicious_df['Proto']=='udp')) & (malicious_df['is_test']==True)]
print(len(mal_base))
mal_adv = mal_base.copy(deep=True)

# attacking
max_dur = mal_adv['Duration'].max()
min_dur = mal_adv['Duration'].min()

mal_adv['Duration'] = mal_adv['Duration'] + dur_intensity # we increase the duration in seconds
mal_adv['Duration'] = np.where(mal_adv['Duration'] > max_dur, max_dur, mal_adv['Duration'])
mal_adv['Duration'] = np.where(mal_adv['Duration'] < min_dur, min_dur, mal_adv['Duration'])


mal_adv['DstPkts'] = mal_adv['DstPkts'] + pkt_intensity
mal_adv['DstBytes'] = mal_adv['DstBytes'] + byt_intensity

#mal_adv['SrcBytes'] = mal_adv['SrcBytes'] + pkt_intensity
#mal_adv['SrcPkts'] = mal_adv['SrcPkts'] + byt_intensity

mal_adv['totPkt'] = mal_adv['SrcPkts'] + mal_adv['DstPkts']
mal_adv['totByt'] = mal_adv['SrcBytes'] + mal_adv['DstBytes']

2616


# FROM NOW ON, THE CODE IS ALWAYS THE SAME FOR EVERY DATASET!!!!!!

# Baseline: Assessment on "Complete" feature set

## BINARY CLASSIFIER (Complete features)

In [8]:
bClf, bPred, bResult = develop_clf(all_train, all_test, features, clf_name='bin', label='Nature', clf_type=base_clf, verbose=1)

if (bResult.acc == 0):
    bErr = int(len(all_test) * (1-bResult.acc_multi))
else:
    bErr = int(len(all_test) * (1-bResult.acc))

print("Total Misclassifications: {} out of {} (Recall: {:5f}\tFPR: {:5f})".format(bErr, len(all_test), bResult.rec, bResult.fpr))
pd.crosstab(all_test['Nature'], bPred, rownames=['True'], colnames=['Pred'])

Training and testing bin......done! Training time: 0.007973s	Inference time: 0.045846s
Total Misclassifications: 2108 out of 121123 (Recall: 0.999101	FPR: 0.020890)


Pred,0,1
True,,
0,97911,2089
1,19,21104


## MULTI-CLASS CLASSIFIER - cascade

In [9]:
# This is the classifier that analyzes ONLY the malicious samples that "receives" from the initial binary classifier
# It is trained on the same training set---but without using the benign samples
# It is tested on the malicious samples in the test set that are flagged as malicious by the binary classifier


mcClf, mcPred, mcResult = develop_clf(malicious_train, malicious_test, features, clf_name='mc', label='Label_cat', clf_type=base_clf, verbose=1)
mcErr = int(len(malicious_test) * (1-mcResult.acc_multi))
print("Total Misclassifications: {} out of {}".format(mcErr, len(malicious_test)))
pd.crosstab(malicious_test['Label_cat'], mcPred, rownames=['True'], colnames=['Pred'])

Training and testing mc......done! Training time: 0.010925s	Inference time: 0.009966s
Total Misclassifications: 9710 out of 21123


Pred,1,2,3,4,5,6,7
True,,,,,,,
1,4111,586,1462,187,365,1159,1035
2,136,2274,109,0,35,140,103
3,648,144,778,63,197,741,699
4,0,0,2,280,18,2,0
5,213,205,250,74,3584,281,242
6,54,15,81,6,42,160,107
7,45,16,96,0,14,138,226


In [10]:
# We select the samples flagged as malicious by the initial classifier.
# Of course, samples flagged as malicious that are NOT actually malicious will always be misclassified

all_test['bPred'] = bPred
mc_test = all_test[(all_test['bPred']==1) & (all_test['Nature']==1)]
if (len(mc_test)==0):
    # in this case, this classifier receives nothing
    print("There is no malicious sample flagged as malicious to analyze!")

mcPred_m = mcClf.predict(mc_test[features])
mcResult.acc_multic = accuracy_score(mc_test['Label_cat'], mcPred_m, normalize=True, sample_weight=None)
mcErr_m = int((1-mcResult.acc_multic) * len(mc_test))
print("Total Misclassifications (among the malicious samples): {} out of {}".format(mcErr_m, len(mc_test)))




pd.crosstab(mc_test['Label_cat'], mcPred_m, rownames=['True'], colnames=['Pred'])

Total Misclassifications (among the malicious samples): 9702 out of 21104


Pred,1,2,3,4,5,6,7
True,,,,,,,
1,4109,586,1461,187,365,1159,1035
2,136,2273,109,0,34,140,103
3,648,142,778,62,195,741,699
4,0,0,2,280,18,2,0
5,213,204,250,74,3576,281,242
6,54,15,81,6,42,160,107
7,45,16,96,0,14,138,226


In [11]:
## Note: We also accounted for the false positives of the first binary classifier (all of which have been considered as misclassifications)
bin_falsePositives = int(bResult.fpr * len(benign_test))
print("This classifier also analyzed {} benign samples that were incorrectly labelled as 'malicious' by the binary classifier".format(bin_falsePositives))

print("Hence, this classifier was tested on {} samples, of which {} have been misclassified".format(len(mc_test)+bin_falsePositives, bin_falsePositives+mcErr_m))

This classifier also analyzed 2088 benign samples that were incorrectly labelled as 'malicious' by the binary classifier
Hence, this classifier was tested on 23192 samples, of which 11790 have been misclassified


## MULTI-CLASS CLASSIFIER - stand-alone

In [12]:
# We first assess its multiclassification performance, and then its binary classification performance

mClf, mPred, mResult = develop_clf(all_train, all_test, features, clf_name='m', label='Label_cat', clf_type=base_clf, verbose=1)
mErr = int(len(all_test) * (1-mResult.acc_multi))
print("Total Misclassifications: {} out of {}".format(mErr, len(all_test)))
mResult.ctab

Training and testing m......done! Training time: 0.011974s	Inference time: 0.042842s
Total Misclassifications: 12177 out of 121123


Pred,0,1,2,3,4,5,6,7
True,,,,,,,,
0,97932,251,274,43,131,1153,32,184
1,5,3859,559,1691,185,374,1243,989
2,1,161,2240,111,0,43,139,102
3,1,642,135,799,61,192,748,692
4,0,0,0,1,279,19,3,0
5,65,247,332,211,74,3454,236,230
6,0,56,13,76,6,43,162,109
7,0,46,16,96,0,16,140,221


In [13]:
# For the binary classification performance, we use the previous predictions
mPred_bin = np.copy(mPred)
mPred_bin[mPred_bin > 0] = 1
mResult.bin_results(all_test['Nature'], mPred_bin)
mErr_bin = int(len(all_test) * (1-mResult.acc))

print("Total Misclassifications: {} out of {} (Recall: {:5f}\tFPR: {:5f})".format(mErr_bin, len(all_test), mResult.rec, mResult.fpr))
mResult.ctab_bin

Total Misclassifications: 2139 out of 121123 (Recall: 0.996591	FPR: 0.020680)


Pred,0,1
True,,
0,97932,2068
1,72,21051


## ENSEMBLE CLASSIFIERS

### Training "individual" binary classifiers

In [14]:
ensemble_df = pd.DataFrame()

benign_train = benign_df[benign_df['is_train']==True]
benign_test = benign_df[benign_df['is_test']==True]


ens_time = 0
ens_avgFPR = 0
tot_TP = 0
tot_P = 0
ens_infer_time = 0
fakeEns_infer_time = 0

for a in attack_names:
    exec(f"{a}_train = {a}_df[{a}_df['is_train']==True]")
    exec(f"{a}_test = {a}_df[{a}_df['is_test']==True]")

    exec(f"train = pd.concat([benign_train, {a}_train])")
    exec(f"test = pd.concat([benign_test, {a}_test])")
    
    # We first train a classifier only on "benign" or on malicious samples of a specific attack. 
    # Afterwards, we immediately test it on a test-set having ONLY malicious samples of this specific attack
    # Note: such "testing" is redundant, because it assumes that the classifier only receives the samples of the attack it is trained on!
    exec(f"{a}Clf, {a}Pred, {a}Result = develop_clf(train, test, features, clf_name='{a}', clf_type=base_clf, verbose=1)")
    exec(f"fakeEns_infer_time += {a}Result.infer_time")
    # We now test the specific classifier on the ENTIRE test-set, thereby allowing to assess its performance also against malicious samples of different attacks
    exec(f"{a}_allPred, {a}_allResults, {a}Result.infer_time = evaluate_clf({a}Clf, all_test, features, clf_name='{a}', time={a}Result.time, verbose=1)")

    exec(f"ensemble_df['{a}'] = {a}_allPred")

    exec(f"tot_TP += ({a}Result.rec * len({a}_test))")
    exec(f"tot_P += len({a}_test)")


    exec(f"ens_avgFPR += {a}Result.fpr")
    exec(f"ens_time += {a}Result.time")
    exec(f"ens_infer_time +={a}Result.infer_time")

ens_avgFPR = ens_avgFPR / len(attack_names)
ens_avgREC = tot_TP/tot_P

print("Total training time: {:5f}s\t AvgFPR: {:5f}\t AvgTPR: {:5f}\tTotal inference time: {:5f}s (fake: {:5f}s)".
      format(ens_time, ens_avgFPR, ens_avgREC, ens_infer_time, fakeEns_infer_time))

Training and testing expl......done! Training time: 0.004995s	Inference time: 0.035875s
Testing expl...
...done! 	Inference time: 0.038402s
Training and testing recon......done! Training time: 0.004944s	Inference time: 0.032881s
Testing recon...
...done! 	Inference time: 0.038872s
Training and testing dos......done! Training time: 0.004967s	Inference time: 0.033863s
Testing dos...
...done! 	Inference time: 0.035879s
Training and testing shell......done! Training time: 0.004982s	Inference time: 0.031863s
Testing shell...
...done! 	Inference time: 0.035387s
Training and testing fuzz......done! Training time: 0.005053s	Inference time: 0.032867s
Testing fuzz...
...done! 	Inference time: 0.035883s
Training and testing bdoor......done! Training time: 0.004985s	Inference time: 0.031964s
Testing bdoor...
...done! 	Inference time: 0.037874s
Training and testing ana......done! Training time: 0.004984s	Inference time: 0.034883s
Testing ana...
...done! 	Inference time: 0.040860s
Total training tim

### Ensemble (real assessment)

In [15]:
# Here we measure the combined performance of the entire ensemble
# This is done with a logical or, or for majority voting (regulated by the "agreement" variable)

ensemble_df["sum"] = ensemble_df.sum(axis=1)
#calculating 
ensemble_df["LOR"] = (ensemble_df["sum"]>0)

#Appending Ground Truth
temp = all_test['Nature'] #> 0)
ensemble_df['True'] = ((temp.reset_index(drop=True)) > 0)

### Ensemble: Logical OR

In [16]:
enslorResult = Result(ensemble_df['True'], ensemble_df['LOR'], ens_time, ens_infer_time)
enslorErr= int(len(all_test) * (1-enslorResult.acc))

print("Total Misclassifications: {} out of {} (Recall: {:5f}\tFPR: {:5f})".format(enslorErr, len(all_test), enslorResult.rec, enslorResult.fpr))
enslorResult.ctab_bin # you can also try with enslorResult.ctab

Total Misclassifications: 2381 out of 121123 (Recall: 0.999479	FPR: 0.023700)


Pred,False,True
True,,
False,97630,2370
True,11,21112


### Ensemble: Majority Voting

In [17]:
min_agree = math.ceil(agreement * len(attack_names))
print("Voting: at least {} out of {} classifiers must agree that a sample is malicious.".format(min_agree, len(attack_names)))
ensemble_df["MAJV"] = (ensemble_df["sum"]>=min_agree)
ensvotResult = Result(ensemble_df['True'], ensemble_df['MAJV'], ens_time, ens_infer_time)
ensvotErr = int(len(all_test) * (1-ensvotResult.acc))
print("Total Misclassifications: {} out of {} (Recall: {:5f}\tFPR: {:5f})".format(ensvotErr, len(all_test), ensvotResult.rec, ensvotResult.fpr))
ensvotResult.ctab_bin # you can also try with ensvotResult.ctab

Voting: at least 4 out of 7 classifiers must agree that a sample is malicious.
Total Misclassifications: 3523 out of 121123 (Recall: 0.886332	FPR: 0.011220)


Pred,False,True
True,,
False,98878,1122
True,2401,18722


### Ensemble: Stacked Classifier

In [18]:
from mlxtend.classifier import StackingClassifier
clf_list = []
for a in attack_names:
    exec(f"clf_list.append({a}Clf)")

meta = choose_clf(clf_type=base_clf)
sClf = StackingClassifier(classifiers=clf_list, meta_classifier=meta, fit_base_estimators=False, use_probas = False)
s_timeStart = time.time()
sClf.fit(all_train[features], all_train['Nature'])
s_time = time.time() - s_timeStart + ens_time
s_timeStart = time.time()
sPred = sClf.predict(all_test[features])
s_infer_time = time.time()-s_timeStart
sResult = Result(all_test['Nature'], sPred, s_time, s_infer_time)
if sResult.acc < sResult.acc_multi:
    sResult.acc = sResult.acc_multi
sErr = int(len(all_test) * (1-sResult.acc))
print("Total Misclassifications: {} out of {} (Recall: {:5f}\tFPR: {:5f})".format(sErr, len(all_test), sResult.rec, sResult.fpr))

sResult.ctab_bin # you can also try with sResult.ctab

Total Misclassifications: 2127 out of 121123 (Recall: 0.998911	FPR: 0.021050)


Pred,0,1
True,,
0,97895,2105
1,23,21100


## Open World Assessment: One attack against all classifiers

In [19]:
### The following code is a mixture of everything described insofar.
### We are only focused on TPR and FPR here. We do not care about accuracy, adversarial robustness, or runtime.
### These experiments are also done only on the "Complete" feature set


oaac_bin_rec = 0
oaac_bin_fpr = 0
oaac_multi_rec = 0
oaac_multi_fpr = 0
oaac_enslor_rec = 0
oaac_enslor_fpr = 0
oaac_ensvot_rec = 0
oaac_ensvot_fpr = 0
oaac_ensstk_rec = 0
oaac_ensstk_fpr = 0

for u in attack_names: # u is the unknown attack
    #print(u) # this is the unknown attack
    #exec(f"{u}_test = {u}_df[{u}_df['is_train']==False]") # create test set by putting the "test" samples of u
    exec(f"{u}_test = pd.concat([benign_test, {u}_test])") # add to the test set the "benign" test samples
    exec(f"{u}_train = benign_df[benign_df['is_train']==True]") # compose the "training" set: start by putting the benign "training" samples
    for a in attack_names: 
        # for every attack that is not u, add its training samples to the training set of u
        if a==u:
            continue
        exec(f"{u}_train = pd.concat([{u}_train, {a}_df[{a}_df['is_train']==True]])")


    # We have created the training and testing set. Now we must train and test a binary classifier by following the standard procedure
    ########## BINARY CLASSIFIER ##########
    exec(f"{u}_oaac_bClf, {u}_oaac_bPred, {u}_oaac_bResult = develop_clf({u}_train, {u}_test, features, clf_name='{u}_oaac_bin', label='Nature', clf_type=base_clf)")


    ########## Multiclass CLASSIFIER ########## --> Train, then test only on binary
    exec(f"{u}_oaac_mClf, {u}_oaac_mPred, {u}_oaac_mResult = develop_clf({u}_train, {u}_test, features, clf_name='{u}_oaac_multi', label='Label_cat', clf_type=base_clf)")
    exec(f"{u}_oaac_mPred_bin = np.copy({u}_oaac_mPred)")
    exec(f"{u}_oaac_mPred_bin[{u}_oaac_mPred_bin > 0] = 1")
    exec(f"{u}_oaac_mResult.bin_results({u}_test['Nature'], {u}_oaac_mPred_bin)")


    ######### Ensemble ##############
    # send the samples in TEST to all the classifiers of the ensemble (which are already trained), aside from the one focusing on u
    exec(f"{u}_oaac_ens_df = pd.DataFrame()")
    for a in attack_names:    
        if a==u:
                continue
        exec(f"{a}_{u}Pred, {a}_{u}Results, {a}_{u}_infer_time = evaluate_clf({a}Clf, {u}_test, features, clf_name='{a}_{u}', time={a}Result.time)")
        exec(f"{u}_oaac_ens_df['{a}'] = {a}_{u}Pred")

    # now we have the dataframe with all the predictions, let's see the aggregate results
    exec(f"{u}_oaac_ens_df['sum'] = {u}_oaac_ens_df.sum(axis=1)")
    exec(f"{u}_oaac_ens_df['LOR'] = ({u}_oaac_ens_df['sum']>0)")
    exec(f"temp = {u}_test['Nature'] #> 0)")
    exec(f"{u}_oaac_ens_df['True'] = ((temp.reset_index(drop=True)) > 0)")
    exec(f"{u}_oaac_enslorResult = Result({u}_oaac_ens_df['True'], {u}_oaac_ens_df['LOR'], (ens_time-{u}Result.time), (ens_infer_time-{u}Result.infer_time))")

    # now we consider the majority voting of the ensemble
    exec(f"{u}_oaac_ens_df['MAJV'] = ({u}_oaac_ens_df['sum']>=min_agree)")
    exec(f"{u}_oaac_ensvotResult = Result({u}_oaac_ens_df['True'], {u}_oaac_ens_df['MAJV'], (ens_time-{u}Result.time), (ens_infer_time-{u}Result.infer_time))")

    # finally, let's use the stacking ensemble
    exec(f"{u}_clf_list = []")
    for a in attack_names:
        if a==u:
                continue
        exec(f"{u}_clf_list.append({a}Clf)")
    exec(f"{u}_oaac_sClf = StackingClassifier(classifiers={u}_clf_list, meta_classifier=meta, fit_base_estimators=False, use_probas = False)")
    exec(f"{u}_oaac_sClf.fit({u}_train[features], {u}_train['Nature'])")
    exec(f"{u}_oaac_sPred = {u}_oaac_sClf.predict({u}_test[features])")
    exec(f"{u}_oaac_sResult = Result({u}_test['Nature'], {u}_oaac_sPred, (ens_time-{u}Result.time), (ens_infer_time-{u}Result.infer_time))")

    # Updating results
    exec(f"oaac_bin_rec += {u}_oaac_bResult.rec")
    exec(f"oaac_bin_fpr += {u}_oaac_bResult.fpr")
    exec(f"oaac_multi_rec += {u}_oaac_mResult.rec")
    exec(f"oaac_multi_fpr += {u}_oaac_mResult.fpr")
    exec(f"oaac_enslor_rec += {u}_oaac_enslorResult.rec")
    exec(f"oaac_enslor_fpr += {u}_oaac_enslorResult.fpr")
    exec(f"oaac_ensvot_rec += {u}_oaac_ensvotResult.rec")
    exec(f"oaac_ensvot_fpr += {u}_oaac_ensvotResult.fpr")
    exec(f"oaac_ensstk_rec += {u}_oaac_sResult.rec")
    exec(f"oaac_ensstk_fpr += {u}_oaac_sResult.fpr")

# Finalizing averages
oaac_bin_rec /= len(attack_names)
oaac_bin_fpr /= len(attack_names)
oaac_multi_rec /= len(attack_names)
oaac_multi_fpr /= len(attack_names)
oaac_enslor_rec /= len(attack_names)
oaac_enslor_fpr /= len(attack_names)
oaac_ensvot_rec /= len(attack_names)
oaac_ensvot_fpr /= len(attack_names)
oaac_ensstk_rec /= len(attack_names)
oaac_ensstk_fpr /= len(attack_names)


print('''Open World assessment: performance against one unknown attack (averaged for all attacks in the dataset)
      Binary CLF: TPR={:5f}\tFPR={:5f}
      Multiclass (binarized) CLF: TPR={:5f}\tFPR={:5f}
      EnsLOR CLF: TPR={:5f}\tFPR={:5f}
      EnsVOT CLF: TPR={:5f}\tFPR={:5f}
      EnsSTK CLF: TPR={:5f}\tFPR={:5f}
      '''.format(oaac_bin_rec, oaac_bin_fpr,
                 oaac_multi_rec, oaac_multi_fpr,
                 oaac_enslor_rec, oaac_enslor_fpr,
                 oaac_ensvot_rec, oaac_ensvot_fpr,
                 oaac_ensstk_rec, oaac_ensstk_fpr))

Open World assessment: performance against one unknown attack (averaged for all attacks in the dataset)
      Binary CLF: TPR=0.980466	FPR=0.019947
      Multiclass (binarized) CLF: TPR=0.973689	FPR=0.020686
      EnsLOR CLF: TPR=0.996930	FPR=0.023234
      EnsVOT CLF: TPR=0.841133	FPR=0.009054
      EnsSTK CLF: TPR=0.984115	FPR=0.020764
      


## Assessment on Essential feature set (and Adversarial attacks)

### BINARY CLASSIFIER (Essential features)

In [20]:
sma_bClf, sma_bPred, sma_bResult = develop_clf(all_train, all_test, small_features, clf_name='adv_bin', label='Nature', clf_type=base_clf, verbose=1)
sma_bErr = int(len(all_test) * (1-sma_bResult.acc))
if (sma_bResult.acc == 0):
    sma_bErr = int(len(all_test) * (1-sma_bResult.acc_multi))
else:
    sma_bErr = int(len(all_test) * (1-sma_bResult.acc))

print("Total Misclassifications: {} out of {} (Recall: {:5f}\tFPR: {:5f})".format(sma_bErr, len(all_test), sma_bResult.rec, sma_bResult.fpr))
pd.crosstab(all_test['Nature'], sma_bPred, rownames=['True'], colnames=['Pred'])

Training and testing adv_bin......done! Training time: 0.005636s	Inference time: 0.030461s
Total Misclassifications: 2107 out of 121123 (Recall: 0.995597	FPR: 0.020140)


Pred,0,1
True,,
0,97986,2014
1,93,21030


#### Adversarial Attack against Binary Classifier

In [21]:
# Note that the adversarial attacks only affect a subset of the initial set of malicious samples
# Hence, we compute the classification performance also on this subset for a far comparison



adv_bPred_base = sma_bClf.predict(mal_base[small_features])
adv_bPred_adv = sma_bClf.predict(mal_adv[small_features])
adv_bin_base_rec =  recall_score(mal_base['Nature'], adv_bPred_base, pos_label=1)
adv_bin_adv_rec = recall_score(mal_adv['Nature'], adv_bPred_adv, pos_label=1)

print("Adversarial Recall (baseline): {:.3f}".format(adv_bin_base_rec))
print("Adversarial Recall (attack): {:.3f}".format(adv_bin_adv_rec))

Adversarial Recall (baseline): 0.985
Adversarial Recall (attack): 1.000


### Multiclass Classifier - cascade (essential feature set)

In [22]:
sma_mcClf, sma_mcPred, sma_mcResult = develop_clf(malicious_train, malicious_test, small_features, clf_name='sma_mc', label='Label_cat', clf_type=base_clf, verbose=1)
sma_mcErr = int(len(malicious_test) * (1-sma_mcResult.acc_multi))

print("Total Misclassifications: {} out of {}".format(sma_mcErr, len(malicious_test)))
pd.crosstab(malicious_test['Label_cat'], sma_mcPred, rownames=['True'], colnames=['Pred'])

Training and testing sma_mc......done! Training time: 0.006935s	Inference time: 0.006977s
Total Misclassifications: 9967 out of 21123


Pred,1,2,3,4,5,6,7
True,,,,,,,
1,3984,587,1571,181,392,1110,1080
2,119,2186,118,0,141,121,112
3,585,208,821,57,201,695,703
4,0,0,1,286,13,2,0
5,282,425,163,19,3517,257,186
6,54,26,86,14,30,143,112
7,44,15,108,0,18,131,219


In [23]:
# We select the samples flagged as malicious by the initial classifier.
# Of course, samples flagged as malicious that are NOT actually malicious will always be misclassified

all_test['sma_bPred'] = sma_bPred
sma_mc_test = all_test[(all_test['sma_bPred']==1) & (all_test['Nature']==1)]
if (len(sma_mc_test)==0):
    # in this case, this classifier receives nothing
    print("There is no malicious sample flagged as malicious to analyze!")

sma_mcPred_m = sma_mcClf.predict(sma_mc_test[small_features])
sma_mcResult.acc_multic = accuracy_score(sma_mc_test['Label_cat'], sma_mcPred_m, normalize=True, sample_weight=None)
sma_mcErr_m = int((1-sma_mcResult.acc_multic) * len(sma_mc_test))
print("Total Misclassifications (among the malicious samples): {} out of {}".format(sma_mcErr_m, len(sma_mc_test)))




pd.crosstab(sma_mc_test['Label_cat'], sma_mcPred_m, rownames=['True'], colnames=['Pred'])

Total Misclassifications (among the malicious samples): 9904 out of 21030


Pred,1,2,3,4,5,6,7
True,,,,,,,
1,3975,577,1561,181,385,1101,1079
2,117,2180,118,0,140,121,112
3,573,201,815,57,200,693,703
4,0,0,1,286,13,2,0
5,282,425,162,19,3511,257,186
6,54,26,86,14,30,140,112
7,44,15,108,0,18,131,219


In [24]:
## Note: We also accounted for the false positives of the first binary classifier (all of which have been considered as misclassifications)
sma_bin_falsePositives = int(sma_bResult.fpr * len(benign_test))
print("This classifier also analyzed {} benign samples that were incorrectly labelled as 'malicious' by the (small) binary classifier".format(sma_bin_falsePositives))

print("Hence, this (small) classifier was tested on {} samples, of which {} have been misclassified".format(len(sma_mc_test)+sma_bin_falsePositives, sma_bin_falsePositives+sma_mcErr_m))

This classifier also analyzed 2014 benign samples that were incorrectly labelled as 'malicious' by the (small) binary classifier
Hence, this (small) classifier was tested on 23044 samples, of which 11918 have been misclassified


### Multiclass Classifier - stand-alone (essential feature set)

In [25]:
sma_mClf, sma_mPred, sma_mResult = develop_clf(all_train, all_test, small_features, clf_name='sma_m', label='Label_cat', clf_type=base_clf, verbose=1)
sma_mErr = int(len(all_test) * (1-sma_mResult.acc_multi))

print("Total Misclassifications: {} out of {}".format(sma_mErr, len(all_test)))
sma_mResult.ctab

Training and testing sma_m......done! Training time: 0.007981s	Inference time: 0.025906s
Total Misclassifications: 12528 out of 121123


Pred,0,1,2,3,4,5,6,7
True,,,,,,,,
0,97875,68,126,219,117,892,465,238
1,43,3673,549,1780,181,408,1146,1125
2,6,102,2176,117,0,150,138,108
3,32,535,189,813,57,194,731,719
4,0,5,0,1,287,8,1,0
5,105,278,421,172,19,3404,245,205
6,4,46,22,86,14,40,152,101
7,0,43,12,106,0,20,140,214


#### Multiclass Classifier: Binary

In [26]:
########## MULTI-CLASS CLASSIFIER - BINARY ##########
sma_mPred_bin = np.copy(sma_mPred)
sma_mPred_bin[sma_mPred_bin > 0] = 1
sma_mResult.bin_results(all_test['Nature'], sma_mPred_bin)
sma_mErr_bin = int(len(all_test) * (1-sma_mResult.acc))

print("Total Misclassifications: {} out of {} (Recall: {:5f}\tFPR: {:5f})".format(sma_mErr_bin, len(all_test), sma_mResult.rec, sma_mResult.fpr))
sma_mResult.ctab_bin


Total Misclassifications: 2315 out of 121123 (Recall: 0.991005	FPR: 0.021250)


Pred,0,1
True,,
0,97875,2125
1,190,20933


#### Adversarial Attack against the Multiclass Classifier (binary)

In [27]:
adv_mPred_base = sma_mClf.predict(mal_base[small_features])
adv_mPred_adv = sma_mClf.predict(mal_adv[small_features])

adv_mPred_base_bin = np.copy(adv_mPred_base)
adv_mPred_base_bin[adv_mPred_base_bin > 0] = 1

adv_mPred_adv_bin = np.copy(adv_mPred_adv)
adv_mPred_adv_bin[adv_mPred_adv_bin > 0] = 1


adv_multi_base_rec =  recall_score(mal_base['Nature'], adv_mPred_base_bin)
adv_multi_adv_rec = recall_score(mal_adv['Nature'], adv_mPred_adv_bin)

print("Adversarial Recall (baseline): {:.3f}".format(adv_multi_base_rec))
print("Adversarial Recall (attack): {:.3f}".format(adv_multi_adv_rec))

Adversarial Recall (baseline): 0.981
Adversarial Recall (attack): 0.998


## Ensemble Classifiers (essential feature set)

In [28]:
sma_ensemble_df = pd.DataFrame()
adv_ensemble_df_base = pd.DataFrame()
adv_ensemble_df_adv = pd.DataFrame()

benign_train = benign_df[benign_df['is_train']==True]
benign_test = benign_df[benign_df['is_test']==True]


sma_ens_time = 0
sma_ens_avgFPR = 0
sma_tot_TP = 0
sma_tot_P = 0
sma_ens_infer_time = 0
sma_fakeEns_infer_time = 0

for a in attack_names:
    exec(f"{a}_train = {a}_df[{a}_df['is_train']==True]")
    exec(f"{a}_test = {a}_df[{a}_df['is_test']==True]")

    exec(f"train = pd.concat([benign_train, {a}_train])")
    exec(f"test = pd.concat([benign_test, {a}_test])")

    exec(f"sma_{a}Clf, sma_{a}Pred, sma_{a}Result = develop_clf(train, test, small_features, clf_name='sma_{a}', clf_type=base_clf, verbose=1)")
    exec(f"sma_fakeEns_infer_time += sma_{a}Result.infer_time")
    
    exec(f"sma_{a}_allPred, sma_{a}_allResults, sma_{a}Result.infer_time = evaluate_clf(sma_{a}Clf, all_test, small_features, clf_name='sma_{a}', time=sma_{a}Result.time, verbose=1)")
    exec(f"sma_ensemble_df['{a}'] = sma_{a}_allPred")

    exec(f"adv_{a}Pred_base = sma_{a}Clf.predict(mal_base[small_features])")
    exec(f"adv_{a}Pred_adv = sma_{a}Clf.predict(mal_adv[small_features])")
    exec(f"adv_ensemble_df_base['{a}'] = adv_{a}Pred_base")
    exec(f"adv_ensemble_df_adv['{a}'] = adv_{a}Pred_adv")



    exec(f"sma_tot_TP += (sma_{a}Result.rec * len({a}_test))")
    exec(f"sma_tot_P += len({a}_test)")


    exec(f"sma_ens_avgFPR += sma_{a}Result.fpr")
    exec(f"sma_ens_time += sma_{a}Result.time")
    exec(f"sma_ens_infer_time +=sma_{a}Result.infer_time")



sma_ens_avgFPR = sma_ens_avgFPR / len(attack_names)
sma_ens_avgREC = sma_tot_TP/sma_tot_P


print("Total training time: {:5f}s\t AvgFPR: {:5f}\t AvgTPR: {:5f}\tTotal inference time: {:5f}s (fake: {:5f}s)".
      format(sma_ens_time, sma_ens_avgFPR, sma_ens_avgREC, sma_ens_infer_time, sma_fakeEns_infer_time))

Training and testing sma_expl......done! Training time: 0.004978s	Inference time: 0.021939s
Testing sma_expl...
...done! 	Inference time: 0.021993s
Training and testing sma_recon......done! Training time: 0.003998s	Inference time: 0.018884s
Testing sma_recon...
...done! 	Inference time: 0.023937s
Training and testing sma_dos......done! Training time: 0.003951s	Inference time: 0.020927s
Testing sma_dos...
...done! 	Inference time: 0.021996s
Training and testing sma_shell......done! Training time: 0.003987s	Inference time: 0.019934s
Testing sma_shell...
...done! 	Inference time: 0.020941s
Training and testing sma_fuzz......done! Training time: 0.003996s	Inference time: 0.019928s
Testing sma_fuzz...
...done! 	Inference time: 0.019971s
Training and testing sma_bdoor......done! Training time: 0.004996s	Inference time: 0.018865s
Testing sma_bdoor...
...done! 	Inference time: 0.021889s
Training and testing sma_ana......done! Training time: 0.003993s	Inference time: 0.016943s
Testing sma_ana..

### Computing real Ensemble

In [29]:
sma_ensemble_df["sum"] = sma_ensemble_df.sum(axis=1)
sma_ensemble_df["LOR"] = (sma_ensemble_df["sum"]>0)
temp = all_test['Nature'] 
sma_ensemble_df['True'] = ((temp.reset_index(drop=True)) > 0)

adv_ensemble_df_base["sum"] = adv_ensemble_df_base.sum(axis=1)
adv_ensemble_df_base["LOR"] = (adv_ensemble_df_base["sum"]>0)
temp = mal_base['Nature']
adv_ensemble_df_base['True'] = ((temp.reset_index(drop=True)) > 0)


adv_ensemble_df_adv["sum"] = adv_ensemble_df_adv.sum(axis=1)
adv_ensemble_df_adv["LOR"] = (adv_ensemble_df_adv["sum"]>0)
temp = mal_adv['Nature']
adv_ensemble_df_adv['True'] = ((temp.reset_index(drop=True)) > 0)

### Ensemble: Logical OR (essential feature set)

In [30]:
sma_enslorResult = Result(sma_ensemble_df['True'], sma_ensemble_df['LOR'], sma_ens_time, sma_ens_infer_time)
sma_enslorErr= int(len(all_test) * (1-sma_enslorResult.acc))

print("Total Misclassifications: {} out of {} (Recall: {:5f}\tFPR: {:5f})".format(sma_enslorErr, len(all_test), sma_enslorResult.rec, sma_enslorResult.fpr))
sma_enslorResult.ctab_bin # you can also try with sma_enslorResult.ctab

Total Misclassifications: 3280 out of 121123 (Recall: 0.997065	FPR: 0.032180)


Pred,False,True
True,,
False,96782,3218
True,62,21061


#### Logical OR: Adversarial Attack

In [31]:
adv_enslor_base_rec = recall_score(mal_base['Nature'], adv_ensemble_df_base["LOR"])
adv_enslor_adv_rec = recall_score(mal_adv['Nature'], adv_ensemble_df_adv["LOR"])

print("Adversarial Recall (baseline): {:.3f}".format(adv_enslor_base_rec))
print("Adversarial Recall (attack): {:.3f}".format(adv_enslor_adv_rec))

Adversarial Recall (baseline): 0.980
Adversarial Recall (attack): 0.976


### Ensemble: Majority Voting (essential feature set)

In [32]:
sma_ensemble_df["MAJV"] = (sma_ensemble_df["sum"]>=min_agree)
sma_ensvotResult = Result(sma_ensemble_df['True'], sma_ensemble_df['MAJV'], sma_ens_time, sma_ens_infer_time)
sma_ensvotErr = int(len(all_test) * (1-sma_ensvotResult.acc))

print("Total Misclassifications: {} out of {} (Recall: {:5f}\tFPR: {:5f})".format(sma_ensvotErr, len(all_test), sma_ensvotResult.rec, sma_ensvotResult.fpr))
sma_ensvotResult.ctab_bin # you can also try with sma_ensvotResult.ctab

Total Misclassifications: 2090 out of 121123 (Recall: 0.970601	FPR: 0.014700)


Pred,False,True
True,,
False,98530,1470
True,621,20502


#### Majority Voting: Adversarial Attack

In [33]:
adv_ensemble_df_base["MAJV"] = (adv_ensemble_df_base["sum"]>=min_agree)
adv_ensemble_df_adv["MAJV"] = (adv_ensemble_df_adv["sum"]>=min_agree)

adv_ensvot_base_rec = recall_score(mal_base['Nature'], adv_ensemble_df_base["MAJV"])
adv_ensvot_adv_rec = recall_score(mal_adv['Nature'], adv_ensemble_df_adv["MAJV"])

print("Adversarial Recall (baseline): {:.3f}".format(adv_ensvot_base_rec))
print("Adversarial Recall (attack): {:.3f}".format(adv_ensvot_adv_rec))

Adversarial Recall (baseline): 0.961
Adversarial Recall (attack): 0.961


### Ensemble: Stacked Classifier (essential feature set)

In [34]:
clf_list = []
for a in attack_names:
    exec(f"clf_list.append(sma_{a}Clf)")

sma_sClf = StackingClassifier(classifiers=clf_list, meta_classifier=meta, fit_base_estimators=False, use_probas = False)
s_timeStart = time.time()
sma_sClf.fit(all_train[small_features], all_train['Nature'])
sma_s_time = time.time() - s_timeStart + sma_ens_time
s_timeStart = time.time()
sma_sPred = sma_sClf.predict(all_test[small_features])
sma_s_infer_time = time.time()-s_timeStart
sma_sResult = Result(all_test['Nature'], sma_sPred, sma_s_time, sma_s_infer_time)
if sma_sResult.acc < sma_sResult.acc_multi:
    sma_sResult.acc = sma_sResult.acc_multi
sma_sErr = int(len(all_test) * (1-sma_sResult.acc))

print("Total Misclassifications: {} out of {} (Recall: {:5f}\tFPR: {:5f})".format(sma_sErr, len(all_test), sma_sResult.rec, sma_sResult.fpr))

sma_sResult.ctab_bin # you can also try with sma_sResult.ctab

Total Misclassifications: 2339 out of 121123 (Recall: 0.997017	FPR: 0.022770)


Pred,0,1
True,,
0,97723,2277
1,63,21060


#### Stacked Classifier: Adversarial Attack

In [35]:
adv_sPred_base = sma_sClf.predict(mal_base[small_features])
adv_sPred_adv = sma_sClf.predict(mal_adv[small_features])

adv_ensstk_base_rec =  recall_score(mal_base['Nature'], adv_sPred_base)
adv_ensstk_adv_rec = recall_score(mal_adv['Nature'], adv_sPred_adv)

print("Adversarial Recall (baseline): {:.3f}".format(adv_ensstk_base_rec))
print("Adversarial Recall (attack): {:.3f}".format(adv_ensstk_adv_rec))

Adversarial Recall (baseline): 0.979
Adversarial Recall (attack): 0.975


# DONE!

In [36]:
# You can now inspect the results by referring to the "Result" variables. 

In [37]:
## BASELINE RESULTS (on Complete Feature Set)
print(
    "TPR",
    bResult.rec,
    mResult.rec,
    ens_avgREC,
    enslorResult.rec,
    ensvotResult.rec,
    sResult.rec,
    "\nFPR",
    bResult.fpr,
    mResult.fpr,
    ens_avgFPR,
    enslorResult.fpr,
    ensvotResult.fpr,
    sResult.fpr,
    "\nTraining Time",
    bResult.time,
    mResult.time,
    ens_time,
    enslorResult.time,
    ensvotResult.time,
    sResult.time,
    "\nInference Time",
    bResult.infer_time,
    mResult.infer_time,
    fakeEns_infer_time,
    enslorResult.infer_time,
    ensvotResult.infer_time,
    sResult.infer_time, 
    "\nAccuracy",
    mResult.acc_multi,   # This is the accuracy on the multiclassification
    mResult.acc,         # This is the accuracy on the binary classification
    mcResult.acc_multic, # This is the accuracy on the multiclassification AFTER the output of the binary classifier (it does not account for benign samples, which are false positives)
    mcResult.acc_multi   # This is the accuracy on the multiclassification on the whole test portion of the malicious dataset
)

TPR 0.9991005065568338 0.9965913932680017 0.9902949391658382 0.999479240638167 0.8863324338398901 0.9989111395161672 
FPR 0.020889999999999964 0.02068000000000003 0.012467142857142852 0.023700000000000054 0.011220000000000008 0.021050000000000013 
Training Time 0.00797271728515625 0.011973857879638672 0.03490853309631348 0.03490853309631348 0.03490853309631348 0.0478816032409668 
Inference Time 0.045845985412597656 0.04284214973449707 0.23419570922851562 0.2631566524505615 0.2631566524505615 0.1954200267791748 
Accuracy 0.8994658322531641 0.9823320096100658 0.5402767247915087 0.5403115087818965


In [38]:
## BASELINE RESULTS (on Essential Feature Set)
print(
    "TPR",
    sma_bResult.rec,
    sma_mResult.rec,
    sma_ens_avgREC,
    sma_enslorResult.rec,
    sma_ensvotResult.rec,
    sma_sResult.rec,
    "\nFPR",
    sma_bResult.fpr,
    sma_mResult.fpr,
    sma_ens_avgFPR,
    sma_enslorResult.fpr,
    sma_ensvotResult.fpr,
    sma_sResult.fpr,
    "\nTraining Time",
    sma_bResult.time,
    sma_mResult.time,
    sma_ens_time,
    sma_enslorResult.time,
    sma_ensvotResult.time,
    sma_sResult.time,
    "\nInference Time",
    sma_bResult.infer_time,
    sma_mResult.infer_time,
    sma_fakeEns_infer_time,
    sma_enslorResult.infer_time,
    sma_ensvotResult.infer_time,
    sma_sResult.infer_time, 
    "\nAccuracy",
    sma_mResult.acc_multi,   # This is the accuracy on the multiclassification
    sma_mResult.acc,         # This is the accuracy on the binary classification
    sma_mcResult.acc_multic, # This is the accuracy on the multiclassification AFTER the output of the binary classifier (it does not account for benign samples, which are false positives)
    sma_mcResult.acc_multi   # This is the accuracy on the multiclassification on the whole test portion of the malicious dataset
)

TPR 0.9955972163045022 0.9910050655683378 0.9853240543483407 0.9970648108696681 0.9706007669365146 0.9970174691095015 
FPR 0.020140000000000047 0.02124999999999999 0.014978571428571444 0.032179999999999986 0.014700000000000046 0.022769999999999957 
Training Time 0.005635738372802734 0.007981300354003906 0.029898643493652344 0.029898643493652344 0.029898643493652344 0.04385995864868164 
Inference Time 0.03046107292175293 0.025905847549438477 0.13741827011108398 0.1516585350036621 0.1516585350036621 0.12357854843139648 
Accuracy 0.8965596955161282 0.9808871973118235 0.5290537327627199 0.5281446764190693


In [39]:
## Open World: One attack against all (averaged results)

print('''Open World assessment: performance against one unknown attack (averaged for all attacks in the dataset)
      BD: TPR={:5f}\tFPR={:5f}
      MD (binarized) CLF: TPR={:5f}\tFPR={:5f}
      ED-o: TPR={:5f}\tFPR={:5f}
      ED-v: TPR={:5f}\tFPR={:5f}
      ED-s: TPR={:5f}\tFPR={:5f}
      '''.format(oaac_bin_rec, oaac_bin_fpr,
                 oaac_multi_rec, oaac_multi_fpr,
                 oaac_enslor_rec, oaac_enslor_fpr,
                 oaac_ensvot_rec, oaac_ensvot_fpr,
                 oaac_ensstk_rec, oaac_ensstk_fpr))

Open World assessment: performance against one unknown attack (averaged for all attacks in the dataset)
      BD: TPR=0.980466	FPR=0.019947
      MD (binarized) CLF: TPR=0.973689	FPR=0.020686
      ED-o: TPR=0.996930	FPR=0.023234
      ED-v: TPR=0.841133	FPR=0.009054
      ED-s: TPR=0.984115	FPR=0.020764
      


In [40]:
## Adversarial Attacks 

print(
    "BD (before, after):",
    adv_bin_base_rec,
    adv_bin_adv_rec,
    "\nMD  (before, after):",
    adv_multi_base_rec,
    adv_multi_adv_rec,
    "\nED-o  (before, after):",
    adv_enslor_base_rec,
    adv_enslor_adv_rec, 
    "\nED-v  (before, after):",
    adv_ensvot_base_rec,
    adv_ensvot_adv_rec, 
    "\nED-s  (before, after):",
    adv_ensstk_base_rec, 
    adv_ensstk_adv_rec,
)

BD (before, after): 0.985474006116208 1.0 
MD  (before, after): 0.9805045871559633 0.9977064220183486 
ED-o  (before, after): 0.9797400611620795 0.9759174311926605 
ED-v  (before, after): 0.9613914373088684 0.9613914373088684 
ED-s  (before, after): 0.9793577981651376 0.9751529051987767
